# Introduction
This notebook includes simple examples to demonstrate how to tune User Defined Functions with `flaml.tune`.

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with options:
```bash
pip install flaml[notebook]
```

In [ ]:
%pip install flaml[notebook]

# Basic tuning procedure
## 1. A basic tuning example

In [2]:
'''Set a search space'''
from flaml import tune

config_search_space = {
    "x": tune.lograndint(lower=1, upper=100000),
    "y": tune.randint(lower=1, upper=100000)
}  

In [3]:
'''Write a evaluation function'''
import time

def evaluate_config(config: dict):
    """evaluate a hyperparameter configuration"""
    score = (config["x"] - 85000) ** 2 - config["x"] / config["y"]
    # usually the evaluation takes an non-neglible cost
    # and the cost could be related to certain hyperparameters
    # here we simulate this cost by calling the time.sleep() function
    # here we assume the cost is proportional to x
    faked_evaluation_cost = config["x"] / 100000
    time.sleep(faked_evaluation_cost)
    # we can return a single float as a score on the input config:
    # return score
    # or, we can return a dictionary that maps metric name to metric value:
    return {"score": score, "evaluation_cost": faked_evaluation_cost, "constraint_metric": config["x"] * config["y"]}

In [4]:
'''Performs tuning'''
# require: pip install flaml[blendsearch]
analysis = tune.run(
    evaluate_config,  # the function to evaluate a config
    config=config_search_space,  # the search space defined
    metric="score",
    mode="min",  # the optimization mode, "min" or "max"
    num_samples=-1,  # the maximal number of configs to try, -1 means infinite
    time_budget_s=10,  # the time budget in seconds
)

[flaml.tune.tune: 01-09 05:39:40] {486} INFO - Using search algorithm type.
You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2023-01-09 05:39:40,135] A new study created in memory with name: optuna
[flaml.tune.tune: 01-09 05:39:40] {636} INFO - trial 1 config: {'x': 3, 'y': 13184}
[flaml.tune.tune: 01-09 05:39:40] {636} INFO - trial 2 config: {'x': 6134, 'y': 2076}
[flaml.tune.tune: 01-09 05:39:40] {636} INFO - trial 3 config: {'x': 1143, 'y': 74880}
[flaml.tune.tune: 01-09 05:39:40] {636} INFO - trial 4 config: {'x': 5539, 'y': 1}
[flaml.tune.tune: 01-09 05:39:40] {636} INFO - trial 5 config: {'x': 6793, 'y': 16190}
[flaml.tune.tune: 01-09 05:39:40] {636} INFO - trial 6 config: {'x': 220, 'y': 22480}
[flaml.tune.t

In [5]:
'''Investigate results'''
print(analysis.best_result)

{'score': 138344643.26761267, 'evaluation_cost': 0.73238, 'constraint_metric': 7323726762, 'training_iteration': 0, 'config': {'x': 73238, 'y': 99999}, 'config/x': 73238, 'config/y': 99999, 'experiment_tag': 'exp', 'time_total_s': 0.7339465618133545}


## Hierarchical search space 
Hierarchical search space is supported.

In [6]:
'''Set a hierarchical search space'''
gbtree_hp_space = {
    "booster": "gbtree",
    "n_estimators": tune.lograndint(lower=4, upper=64),
    "max_leaves": tune.lograndint(lower=4, upper=64),
    "learning_rate": tune.loguniform(lower=1 / 1024, upper=1.0),
}
gblinear_hp_space = {
    "booster": "gblinear",
    "lambda": tune.uniform(0, 1),
    "alpha": tune.loguniform(0.0001, 1),
}

full_space = {
    "xgb_config": tune.choice([gbtree_hp_space, gblinear_hp_space]),
}

In [7]:
'''Write a evaluation function'''
import xgboost as xgb

def xgb_obj(X_train, X_test, y_train, y_test, config):
    config = config["xgb_config"]
    params = config2params(config)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    booster_type = config.get("booster")

    if booster_type == "gblinear":
        model = xgb.train(
            params,
            dtrain,
        )
    else:
        _n_estimators = params.pop("n_estimators")
        model = xgb.train(params, dtrain, _n_estimators)

    # get validation loss
    from sklearn.metrics import r2_score

    dtest = xgb.DMatrix(X_test)
    y_test_predict = model.predict(dtest)
    test_loss = 1.0 - r2_score(y_test, y_test_predict)
    return {"loss": test_loss}

def config2params(config: dict) -> dict:
    params = config.copy()
    max_depth = params["max_depth"] = params.get("max_depth", 0)
    if max_depth == 0:
        params["grow_policy"] = params.get("grow_policy", "lossguide")
        params["tree_method"] = params.get("tree_method", "hist")
    # params["booster"] = params.get("booster", "gbtree")
    params["use_label_encoder"] = params.get("use_label_encoder", False)
    if "n_jobs" in config:
        params["nthread"] = params.pop("n_jobs")
    return params

In [8]:
'''Tune xgb_obj with configs from the hierarchical search space'''
from flaml.data import load_openml_dataset
from functools import partial

X_train, X_test, y_train, y_test = load_openml_dataset(
    dataset_id=537, data_dir="./"
)
analysis = tune.run(
    partial(xgb_obj, X_train, X_test, y_train, y_test),
    config=full_space,
    metric="loss",
    mode="min",
    num_samples=5,
)
print("analysis", analysis.results)

[flaml.tune.tune: 01-09 05:39:50] {486} INFO - Using search algorithm type.
[I 2023-01-09 05:39:50,983] A new study created in memory with name: optuna
[flaml.tune.tune: 01-09 05:39:50] {636} INFO - trial 1 config: {'xgb_config': {'booster': 'gblinear', 'lambda': 0.6472660813321921, 'alpha': 0.0028264214081400044}}
[flaml.tune.tune: 01-09 05:39:51] {636} INFO - trial 2 config: {'xgb_config': {'n_estimators': 22, 'max_leaves': 31, 'learning_rate': 0.0309282737630552, 'booster': 'gbtree'}}
[flaml.tune.tune: 01-09 05:39:51] {636} INFO - trial 3 config: {'xgb_config': {'n_estimators': 32, 'max_leaves': 6, 'learning_rate': 0.0018014797394283806, 'booster': 'gbtree'}}
[flaml.tune.tune: 01-09 05:39:51] {636} INFO - trial 4 config: {'xgb_config': {'n_estimators': 21, 'max_leaves': 21, 'learning_rate': 0.06308266770250766, 'booster': 'gbtree'}}
[flaml.tune.tune: 01-09 05:39:51] {636} INFO - trial 5 config: {'xgb_config': {'n_estimators': 44, 'max_leaves': 6, 'learning_rate': 0.0009765625, 'boos

load dataset from ./openml_ds537.pkl
Dataset name: houses
X_train.shape: (15480, 8), y_train.shape: (15480,);
X_test.shape: (5160, 8), y_test.shape: (5160,)
[05:39:50] WARNING: ../src/learner.cc:767: 
Parameters: { "grow_policy", "max_depth", "tree_method", "use_label_encoder" } are not used.

[05:39:51] WARNING: ../src/learner.cc:767: 
Parameters: { "use_label_encoder" } are not used.

[05:39:51] WARNING: ../src/learner.cc:767: 
Parameters: { "use_label_encoder" } are not used.

[05:39:51] WARNING: ../src/learner.cc:767: 
Parameters: { "use_label_encoder" } are not used.

[05:39:51] WARNING: ../src/learner.cc:767: 
Parameters: { "use_label_encoder" } are not used.

analysis {'094cf46a': {'loss': 422.4636187077617, 'training_iteration': 0, 'config': {'xgb_config': {'booster': 'gblinear', 'lambda': 0.6472660813321921, 'alpha': 0.0028264214081400044}}, 'config/xgb_config': {'booster': 'gblinear', 'lambda': 0.6472660813321921, 'alpha': 0.0028264214081400044}, 'experiment_tag': 'exp', 'tim

# Advanced Tuning Options

## Parallel tuning

In [ ]:
%pip install flaml[ray]

In [10]:
# require: pip install flaml[ray]
analysis = tune.run(
    evaluate_config,  # the function to evaluate a config
    config=config_search_space,  # the search space defined
    metric="score",
    mode="min",  # the optimization mode, "min" or "max"
    num_samples=-1,  # the maximal number of configs to try, -1 means infinite
    time_budget_s=10,  # the time budget in seconds
    use_ray=True,
    resources_per_trial={"cpu": 2}  # limit resources allocated per trial
)
print(analysis.best_trial.last_result)  # the best trial's result
print(analysis.best_config)  # the best config

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2023-01-09 05:39:53,893] A new study created in memory with name: optuna
2023-01-09 05:39:55,654	WARNING services.py:2002 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-01-09 05:39:56,961	WARNING function_runner.py:603 -- Function checkpointing is disabled. This may result i

Trial evaluate_config_0d564278 reported score=7224490009.00 with parameters={'x': 3, 'y': 13184}.
Trial evaluate_config_0d564278 completed. Last result: score=7224490008.999772,evaluation_cost=3e-05,constraint_metric=39552
Trial evaluate_config_0e3e5fb8 reported score=7031996448.98 with parameters={'x': 1143, 'y': 74880}.
Trial evaluate_config_0e3e5fb8 completed. Last result: score=7031996448.9847355,evaluation_cost=0.01143,constraint_metric=85587840
Trial evaluate_config_0e434d5c reported score=6864453903.98 with parameters={'x': 2148, 'y': 95339}.
Trial evaluate_config_0e434d5c completed. Last result: score=6864453903.977469,evaluation_cost=0.02148,constraint_metric=204788172
Trial evaluate_config_0e4bb1f4 reported score=2337335715.49 with parameters={'x': 36654, 'y': 71457}.
Trial evaluate_config_0e4bb1f4 completed. Last result: score=2337335715.487048,evaluation_cost=0.36654,constraint_metric=2619184878
Trial evaluate_config_0e51503c reported score=7161221375.97 with parameters={'x

Trial evaluate_config_1048d482 completed. Last result: score=3411961743.5586176,evaluation_cost=0.26588,constraint_metric=1601607944
Trial evaluate_config_0fd5a426 reported score=224969999.46 with parameters={'x': 99999, 'y': 65124}.
Trial evaluate_config_0fd5a426 completed. Last result: score=224969999.46448314,evaluation_cost=0.99999,constraint_metric=6512334876
Trial evaluate_config_0ffb44d8 reported score=224969999.79 with parameters={'x': 99999, 'y': 82599}.
Trial evaluate_config_0ffb44d8 completed. Last result: score=224969999.78934368,evaluation_cost=0.99999,constraint_metric=8259817401
Trial evaluate_config_100ef42e reported score=224969999.43 with parameters={'x': 99999, 'y': 63770}.
Trial evaluate_config_100ef42e completed. Last result: score=224969999.4318802,evaluation_cost=0.99999,constraint_metric=6376936230
Trial evaluate_config_104a1b3a reported score=79637774.76 with parameters={'x': 76076, 'y': 61550}.
Trial evaluate_config_104a1b3a completed. Last result: score=79637

2023-01-09 05:40:07,892	INFO stopper.py:363 -- Reached timeout of 10 seconds. Stopping all trials.


Trial evaluate_config_131c635e reported score=1776959715.31 with parameters={'x': 42846, 'y': 61902}.
Trial evaluate_config_131c635e completed. Last result: score=1776959715.3078415,evaluation_cost=0.42846,constraint_metric=2652253092


Trial name,status,loc,x,y,iter,total time (s),score,evaluation_cost,constraint_metric
evaluate_config_0d564278,TERMINATED,223.255.255.2:546754,3,13184,1,0.000300646,7.22449e+09,3e-05,39552
evaluate_config_0e3cc400,TERMINATED,223.255.255.2:546789,6134,2076,1,0.0616293,6.21985e+09,0.06134,12734184
evaluate_config_0e3e5fb8,TERMINATED,223.255.255.2:546754,1143,74880,1,0.0116265,7.032e+09,0.01143,85587840
evaluate_config_0e3fe6c6,TERMINATED,223.255.255.2:546793,220,22480,1,0.00239468,7.18765e+09,0.0022,4945600
evaluate_config_0e40f2be,TERMINATED,223.255.255.2:546795,6,76053,1,0.000358582,7.22398e+09,6e-05,456318
evaluate_config_0e42000a,TERMINATED,223.255.255.2:546798,4,8834,1,0.00026536,7.22432e+09,4e-05,35336
evaluate_config_0e434d5c,TERMINATED,223.255.255.2:546754,2148,95339,1,0.0216658,6.86445e+09,0.02148,204788172
evaluate_config_0e448b4a,TERMINATED,223.255.255.2:546802,1,51219,1,0.000264406,7.22483e+09,1e-05,51219
evaluate_config_0e4582c0,TERMINATED,223.255.255.2:546803,10155,61252,1,0.101861,5.60177e+09,0.10155,622014060
evaluate_config_0e4a307c,TERMINATED,223.255.255.2:546808,3350,29188,1,0.0337877,6.66672e+09,0.0335,97779800


2023-01-09 05:40:08,543	INFO tune.py:747 -- Total run time: 11.58 seconds (10.36 seconds for the tuning loop).


{'score': 2208.0280898234387, 'evaluation_cost': 0.85047, 'constraint_metric': 7442037735, 'time_this_iter_s': 0.8514487743377686, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 1, 'trial_id': '0f2fb034', 'experiment_id': '4f147ebc39e242588d62fc3d1205138f', 'date': '2023-01-09_05-40-01', 'timestamp': 1673242801, 'time_total_s': 0.8514487743377686, 'pid': 546808, 'hostname': '4dd4bf5c6823', 'node_ip': '223.255.255.2', 'config': {'x': 85047, 'y': 87505}, 'time_since_restore': 0.8514487743377686, 'timesteps_since_restore': 0, 'iterations_since_restore': 1, 'warmup_time': 0.0022461414337158203, 'experiment_tag': '27_x=85047,y=87505'}
{'x': 85047, 'y': 87505}


## Warm start

In [11]:
config_search_space = {
    "a": tune.uniform(lower=0, upper=0.99),
    "b": tune.uniform(lower=0, upper=3),
}

def simple_obj(config):
    return config["a"] + config["b"]

points_to_evaluate = [
    {"b": .99, "a": 3},
    {"b": .99, "a": 2},
    {"b": .80, "a": 3},
    {"b": .80, "a": 2},
]
evaluated_rewards = [3.99, 2.99]

analysis = tune.run(
    simple_obj,
    config=config_search_space,
    mode="max",
    points_to_evaluate=points_to_evaluate,
    evaluated_rewards=evaluated_rewards,
    num_samples=10,
)

[flaml.tune.tune: 01-09 05:40:08] {486} INFO - Using search algorithm type.
2023-01-09 05:40:08,948	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2023-01-09 05:40:08,949] A new study created in memory with name: optuna
/home/vscode/.local/lib/python3.9/site-packages/ray/tune/suggest/optuna.py:561: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  trial = ot.trial.create_trial(
2023-01-09 05:40:08,950	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune

## Trial scheduling

###  An authentic scheduler implemented in FLAML (`scheduler='flaml'`).

In [12]:
search_space = {
    "n_estimators": tune.lograndint(lower=4, upper=32768),
    "num_leaves": tune.lograndint(lower=4, upper=32768),
    "learning_rate": tune.loguniform(lower=1 / 1024, upper=1.0),
}

In [13]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

'''Set a evaluation function with resource dimension'''
def obj_from_resource_attr(resource_attr, X_train, X_test, y_train, y_test, config):

    # in this example sample size is our resource dimension
    resource = int(config[resource_attr])
    sampled_X_train = X_train.iloc[:resource]
    sampled_y_train = y_train.iloc[:resource]

    # construct a LGBM model from the config
    # note that you need to first remove the resource_attr field
    # from the config as it is not part of the original search space
    model_config = config.copy()
    del model_config[resource_attr]
    model = LGBMClassifier(**model_config)

    model.fit(sampled_X_train, sampled_y_train)
    y_test_predict = model.predict(X_test)
    test_loss = 1.0 - accuracy_score(y_test, y_test_predict)
    return {"loss": test_loss}

In [26]:
from flaml.data import load_openml_task
from sklearn.utils import shuffle

X_train, X_test, y_train, y_test = load_openml_task(task_id=7592, data_dir="")
# shuffle X_train and y_train
X_train, y_train = shuffle(X_train, y_train)
max_resource = len(y_train)
resource_attr = "sample_size"
min_resource = 10000
analysis = tune.run(
    partial(
        obj_from_resource_attr, resource_attr, X_train, X_test, y_train, y_test
    ),
    config=search_space,
    metric="loss",
    mode="min",
    resource_attr=resource_attr,
    scheduler="flaml",
    max_resource=max_resource,
    min_resource=min_resource,
    time_budget_s=300,
    num_samples=-1,
)
print("best result w/ flaml scheduler (in 300s): ", analysis.best_result)

[flaml.tune.tune: 01-09 06:14:32] {486} INFO - Using search algorithm type.
2023-01-09 06:14:32,337	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2023-01-09 06:14:32,338] A new study created in memory with name: optuna
[flaml.tune.tune: 01-09 06:14:32] {636} INFO - trial 1 config: {'n_estimators': 9, 'num_leaves': 1364, 'learning_rate': 0.012074374674294664, 'sample_size': 10000}
[flaml.tune.tune: 01-09 06:14:32] {636} INFO - trial 2 config: {'n_estimators': 4048, 'num_leaves': 4, 'learning_rate': 0.07891713267442702, 'sample_size': 10000}


load dataset from openml_task7592.pkl
X_train.shape: (43957, 14), y_train.shape: (43957,),
X_test.shape: (4885, 14), y_test.shape: (4885,)


[flaml.tune.tune: 01-09 06:14:34] {636} INFO - trial 3 config: {'n_estimators': 3295, 'num_leaves': 334, 'learning_rate': 0.004638797085780012, 'sample_size': 10000}
[flaml.tune.tune: 01-09 06:14:56] {636} INFO - trial 4 config: {'n_estimators': 3741, 'num_leaves': 4, 'learning_rate': 0.1609628553690032, 'sample_size': 10000}
[flaml.tune.tune: 01-09 06:14:58] {636} INFO - trial 5 config: {'n_estimators': 4380, 'num_leaves': 14, 'learning_rate': 0.03869162121457021, 'sample_size': 10000}
[flaml.tune.tune: 01-09 06:15:02] {636} INFO - trial 6 config: {'n_estimators': 9099, 'num_leaves': 347, 'learning_rate': 0.0018653919541457076, 'sample_size': 10000}
[flaml.tune.tune: 01-09 06:16:09] {636} INFO - trial 7 config: {'n_estimators': 8, 'num_leaves': 1156, 'learning_rate': 0.03978162762775204, 'sample_size': 10000}
[flaml.tune.tune: 01-09 06:16:09] {636} INFO - trial 8 config: {'n_estimators': 21, 'num_leaves': 3668, 'learning_rate': 0.003153366048206083, 'sample_size': 10000}
[flaml.tune.t

best result w/ flaml scheduler (in 300s):  {'loss': 0.12221084953940631, 'training_iteration': 0, 'config': {'n_estimators': 141, 'num_leaves': 4, 'learning_rate': 0.543717371799006, 'sample_size': 43957}, 'config/n_estimators': 141, 'config/num_leaves': 4, 'config/learning_rate': 0.543717371799006, 'config/sample_size': 43957, 'experiment_tag': 'exp', 'time_total_s': 0.32015228271484375}


###  ASHA scheduler (`scheduler='asha'`) or a custom scheduler of the  [`TrialScheduler`](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers) class from `ray.tune`.

In [27]:
def obj_w_intermediate_report(
    resource_attr,
    X_train,
    X_test,
    y_train,
    y_test,
    min_resource,
    max_resource,
    config,
):
    # a customized schedule to perform the evaluation
    eval_schedule = [res for res in range(min_resource, max_resource, 5000)] + [
        max_resource
    ]
    for resource in eval_schedule:
        sampled_X_train = X_train.iloc[:resource]
        sampled_y_train = y_train.iloc[:resource]

        # construct a LGBM model from the config
        model = LGBMClassifier(**config)

        model.fit(sampled_X_train, sampled_y_train)
        y_test_predict = model.predict(X_test)
        test_loss = 1.0 - accuracy_score(y_test, y_test_predict)
        # need to report the resource attribute used and the corresponding intermediate results
        try:
            tune.report(sample_size=resource, loss=test_loss)
        except StopIteration:
            return

In [28]:
X_train, X_test, y_train, y_test = load_openml_task(task_id=7592, data_dir="")
resource_attr = "sample_size"
min_resource = 10000
max_resource = len(y_train)
analysis = tune.run(
    partial(
        obj_w_intermediate_report,
        resource_attr,
        X_train,
        X_test,
        y_train,
        y_test,
        min_resource,
        max_resource,
    ),
    config=search_space,
    metric="loss",
    mode="min",
    resource_attr=resource_attr,
    scheduler="asha",
    max_resource=max_resource,
    min_resource=min_resource,
    time_budget_s=300,
    num_samples=-1,
)
print("best result w/ asha scheduler (in 300s): ", analysis.best_result)

[flaml.tune.tune: 01-09 06:20:34] {486} INFO - Using search algorithm type.
2023-01-09 06:20:34,387	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2023-01-09 06:20:34,388] A new study created in memory with name: optuna
[flaml.tune.tune: 01-09 06:20:34] {636} INFO - trial 1 config: {'n_estimators': 9, 'num_leaves': 1364, 'learning_rate': 0.012074374674294664}


load dataset from openml_task7592.pkl
X_train.shape: (43957, 14), y_train.shape: (43957,),
X_test.shape: (4885, 14), y_test.shape: (4885,)


[flaml.tune.tune: 01-09 06:20:36] {636} INFO - trial 2 config: {'n_estimators': 4048, 'num_leaves': 4, 'learning_rate': 0.07891713267442702}
[flaml.tune.tune: 01-09 06:21:11] {636} INFO - trial 3 config: {'n_estimators': 3295, 'num_leaves': 334, 'learning_rate': 0.004638797085780012}
[flaml.tune.tune: 01-09 06:24:39] {636} INFO - trial 4 config: {'n_estimators': 21, 'num_leaves': 3668, 'learning_rate': 0.003153366048206083}
[flaml.tune.tune: 01-09 06:24:39] {636} INFO - trial 5 config: {'n_estimators': 8, 'num_leaves': 1845, 'learning_rate': 0.7239356970260848}
[flaml.tune.tune: 01-09 06:24:39] {636} INFO - trial 6 config: {'n_estimators': 4, 'num_leaves': 379, 'learning_rate': 0.2728556109672425}
[flaml.tune.tune: 01-09 06:24:39] {636} INFO - trial 7 config: {'n_estimators': 948, 'num_leaves': 2573, 'learning_rate': 0.0073847289359894605}


best result w/ asha scheduler (in 300s):  {'sample_size': 43957, 'loss': 0.12302968270214942, 'training_iteration': 7, 'config': {'n_estimators': 4048, 'num_leaves': 4, 'learning_rate': 0.07891713267442702}, 'config/n_estimators': 4048, 'config/num_leaves': 4, 'config/learning_rate': 0.07891713267442702, 'experiment_tag': 'exp', 'time_total_s': 35.0105664730072}
